In [1]:
# import requests
# import os
# import time
# from urllib.request import urlretrieve
# import zipfile
# import json

# # Constants
# CLIENT_ID = "ShGAtRM5_Wp66hltlbsFaGBu01tTEiMdSEE_HXBv0wI"  # Replace with your actual key
# SEASON = "fall"
# IMAGE_COUNT = {"train": 6000, "validation": 1500, "test": 1500}
# PER_PAGE = 30
# BASE_PATH = "/content/project"
# SECONDS_BETWEEN_REQUESTS = 72  # Throttle requests to stay under rate limit
# PROGRESS_FILE = "/content/download_progress_fall.json"

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# # Function to fetch images
# def get_unsplash_photos(keyword, client_id, page, per_page=PER_PAGE):
#     url = "https://api.unsplash.com/search/photos"
#     params = {"query": keyword, "client_id": client_id, "per_page": per_page, "page": page}
#     response = requests.get(url, params=params)
#     if response.status_code == 200:
#         return [photo['urls']['regular'] for photo in response.json()['results']]
#     else:
#         print(f"Failed to fetch photos for {keyword}, page {page}: {response.status_code}")
#         return []

# # Function to download and save images with custom filenames
# def download_and_save_images(image_urls, folder_path, image_name, start_index):
#     if not os.path.exists(folder_path):
#         os.makedirs(folder_path)
#     for i, url in enumerate(image_urls, start=start_index):
#         filename = f"{folder_path}/{image_name}_{i}.jpg"
#         urlretrieve(url, filename)

# # Function to download images for a season and dataset part
# def download_season_images(season, part, target_count, progress):
#     folder_name = os.path.join(BASE_PATH, season, part)
#     current_page = progress.get(season, {}).get(part, 1)
#     downloaded_count = (current_page - 1) * PER_PAGE
#     start_index = downloaded_count

#     while downloaded_count < target_count:
#         image_urls = get_unsplash_photos(season, CLIENT_ID, current_page)
#         download_and_save_images(image_urls, folder_name, season.capitalize(), start_index)
#         print(f"Successfully download photo {start_index}")
#         downloaded_count += len(image_urls)
#         current_page += 1
#         start_index += len(image_urls)
#         save_progress(season, part, current_page)
#         if downloaded_count < target_count:
#             time.sleep(SECONDS_BETWEEN_REQUESTS)  # Throttle requests


# # Save progress to a file
# def save_progress(season, part, page):
#     if os.path.exists(PROGRESS_FILE):
#         with open(PROGRESS_FILE, 'r') as file:
#             progress = json.load(file)
#     else:
#         progress = {}

#     if season not in progress:
#         progress[season] = {}
#     progress[season][part] = page

#     with open(PROGRESS_FILE, 'w') as file:
#         json.dump(progress, file)

# # Load progress from a file
# def load_progress():
#     if os.path.exists(PROGRESS_FILE):
#         with open(PROGRESS_FILE, 'r') as file:
#             return json.load(file)
#     return {}

# # Zip the folders
# def zip_folders():
#     with zipfile.ZipFile('/content/project.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
#         for root, dirs, files in os.walk(BASE_PATH):
#             for file in files:
#                 zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), BASE_PATH))

In [1]:
import requests
import os
import time
from urllib.request import urlretrieve
import zipfile
import json

# Constants
CLIENT_ID = "ShGAtRM5_Wp66hltlbsFaGBu01tTEiMdSEE_HXBv0wI"  # Replace with your actual key
SEASON = "fall"
IMAGE_COUNT = {"train": 10000, "validation": 2500, "test": 2500}
PER_PAGE = 10  # Number of images per page
SECONDS_BETWEEN_REQUESTS = 1.2  # Throttle requests to avoid hitting rate limit too quickly
BASE_PATH = "/content/project"  # Base directory for saving images
PROGRESS_FILE = "/content/download_progress.json"  # File to track download progress

In [4]:
# Function to fetch images from Unsplash
def get_unsplash_photos(keyword, client_id, page, per_page=PER_PAGE):
    url = "https://api.unsplash.com/search/photos"
    params = {"query": keyword, "client_id": client_id, "per_page": per_page, "page": page}
    response = requests.get(url, params=params)
    if response.status_code == 200:
        return [photo['urls']['regular'] for photo in response.json()['results']]

    else:
        print(f"Failed to fetch photos for {keyword}, page {page}: {response.status_code}")
        return []
# Function to download and save images with custom filenames
def download_and_save_images(image_urls, folder_path, image_name, start_index):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for i, url in enumerate(image_urls, start=start_index):
        filename = f"{folder_path}/{image_name}_{i}.jpg"
        urlretrieve(url, filename)

# Function to download images for a season and dataset part
def download_season_images(season, part, target_count, progress):
    folder_name = os.path.join(BASE_PATH, season, part)
    current_page = progress.get(season, {}).get(part, 1)
    downloaded_count = (current_page - 1) * PER_PAGE

    while downloaded_count < target_count:
        image_urls = get_unsplash_photos(season, CLIENT_ID, current_page)
        if len(image_urls) != []:
          print(f"Successfully downloaded photo {downloaded_count} on page {current_page}")
        download_and_save_images(image_urls, folder_name, season.capitalize(), downloaded_count)
        downloaded_count += len(image_urls)
        current_page += 1
        save_progress(season, part, current_page)
        if downloaded_count < target_count:
            time.sleep(SECONDS_BETWEEN_REQUESTS)  # Throttle requests

# Function to save progress to a file
def save_progress(season, part, page):
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as file:
            progress = json.load(file)
    else:
        progress = {}

    if season not in progress:
        progress[season] = {}
    progress[season][part] = page

    with open(PROGRESS_FILE, 'w') as file:
        json.dump(progress, file)

# Function to load progress from a file
def load_progress():
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, 'r') as file:
            return json.load(file)
    return {}

# Function to zip the folders
def zip_folders():
    with zipfile.ZipFile('/content/project.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(BASE_PATH):
            for file in files:
                zipf.write(os.path.join(root, file), os.path.relpath(os.path.join(root, file), BASE_PATH))

In [5]:
# Main download loop with progress tracking
progress = load_progress()
for part, count in IMAGE_COUNT.items():
    download_season_images(SEASON, part, count, progress)

zip_folders()
print("Zipping complete. Download '/content/project.zip'")

Successfully downloaded photo 0 on 1
Successfully downloaded photo 10 on 2
Successfully downloaded photo 20 on 3
Successfully downloaded photo 30 on 4
Successfully downloaded photo 40 on 5
Successfully downloaded photo 50 on 6
Successfully downloaded photo 60 on 7
Successfully downloaded photo 70 on 8
Successfully downloaded photo 80 on 9
Successfully downloaded photo 90 on 10
Successfully downloaded photo 100 on 11
Successfully downloaded photo 110 on 12
Successfully downloaded photo 120 on 13
Successfully downloaded photo 130 on 14
Successfully downloaded photo 140 on 15
Successfully downloaded photo 150 on 16
Successfully downloaded photo 160 on 17
Successfully downloaded photo 170 on 18
Successfully downloaded photo 180 on 19
Successfully downloaded photo 190 on 20
Successfully downloaded photo 200 on 21
Successfully downloaded photo 210 on 22
Successfully downloaded photo 220 on 23
Successfully downloaded photo 230 on 24
Successfully downloaded photo 240 on 25
Successfully downloa

KeyboardInterrupt: ignored

In [4]:
# Main download loop with progress tracking
progress = load_progress()
for part, count in IMAGE_COUNT.items():
    if part != 'train':  # Skip the training data
        download_season_images(SEASON, part, count, progress)

zip_folders()
print("Zipping complete. Download '/content/project.zip'")

Successfully downloaded photo 0 on 201 page
Successfully downloaded photo 0 on 202 page


KeyboardInterrupt: ignored

In [8]:
image_urls = get_unsplash_photos('fall', CLIENT_ID, 200)
print(image_urls)

['https://images.unsplash.com/photo-1696279892064-f3d2ae451f3a?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w1MzM2NTd8MHwxfHNlYXJjaHw1OTcxfHxmYWxsfGVufDB8fHx8MTcwMTIxMTk5MHww&ixlib=rb-4.0.3&q=80&w=1080', 'https://images.unsplash.com/photo-1699143494459-5d8e398769b7?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w1MzM2NTd8MHwxfHNlYXJjaHw1OTcyfHxmYWxsfGVufDB8fHx8MTcwMTIxMTk5MHww&ixlib=rb-4.0.3&q=80&w=1080', 'https://images.unsplash.com/photo-1700571424825-51a4e0af4014?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w1MzM2NTd8MHwxfHNlYXJjaHw1OTczfHxmYWxsfGVufDB8fHx8MTcwMTIxMTk5MHww&ixlib=rb-4.0.3&q=80&w=1080', 'https://images.unsplash.com/photo-1604877587881-fd3665c9813d?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w1MzM2NTd8MHwxfHNlYXJjaHw1OTc0fHxmYWxsfGVufDB8fHx8MTcwMTIxMTk5MHww&ixlib=rb-4.0.3&q=80&w=1080', 'https://images.unsplash.com/photo-1699244143945-0761e66358a3?crop=entropy&cs=tinysrgb&fit=max&fm=jpg&ixid=M3w1MzM2NTd8MHwxfHNlYXJjaHw1OTc1fHxmYWxsfGVufDB8fHx8MTcwMTIxMTk5MHww&ixl